In [1]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
text = ("After Abraham Lincoln won the November 1860 presidential [MASK] on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy. War broke out in "
        "April 1861 when secessionist forces [MASK] Fort Sumter in South "
        "Carolina, just over a month after Lincoln's inauguration.")

In [4]:
inputs = tokenizer(text, return_tensors='pt')
outputs = model(**inputs)

In [5]:
outputs.keys() # returns MLM output logits

odict_keys(['logits'])

In [6]:
outputs.logits.shape # (1, 46, 30522) - 1 batch, 46 tokens, 30522 vocab size

torch.Size([1, 62, 30522])

In [7]:
# To identify the masked token, we can use the input_ids tensor and find the index of the masked token (103)
mask_position = torch.flatten((inputs.input_ids[0] == 103).nonzero()).tolist()
mask_position # 

[9, 43]

It is for these two positions that we must calculate the loss for when training our model. How does that work? Well, we compare the inputs at those two positions, to the predicted outputs at those two positions - converted to one-hot encoding and probability distribution respectively.

To convert the inputs tokens to one-hot encodings we need the vocab dictionary length, and add a 1 at the position given by the token value.

In [8]:
vocab_size = len(tokenizer.get_vocab())
vocab_size

30522

In [9]:
# Which aligns to our outputs.logits.shape tensor
outputs.logits.shape    

torch.Size([1, 62, 30522])

In [10]:
# In reality, we must mask our tokens randomly, after which we feed the origingal unmasked token ids to the model as labels and keep the token ids as the new masked tensor

In [11]:
# To test this, let's unmask the text first
text = ("After Abraham Lincoln won the November 1860 presidential election on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy. War broke out in "
        "April 1861 when secessionist forces attacked Fort Sumter in South "
        "Carolina, just over a month after Lincoln's inauguration.")

In [12]:
# Then tokenize it
inputs = tokenizer(text, return_tensors='pt')
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,  2602,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [13]:
inputs.input_ids == 102

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False,  True]])

Now we want to mask a number of tokens in the input_ids tensor, the original BERT model was pretrained using a masking probability of 15%, alongside a few additional rules. We will create a similiar, but simpler implementation by taking the primary 15% masking functionality.

In [14]:
# create a random array of floats in equal dimension to input_ids
rand = torch.rand(inputs.input_ids.shape)
rand

tensor([[0.8714, 0.6098, 0.2372, 0.7231, 0.3006, 0.5736, 0.7462, 0.5821, 0.5572,
         0.0954, 0.4532, 0.5451, 0.4395, 0.6812, 0.2518, 0.6894, 0.7780, 0.7485,
         0.0721, 0.3869, 0.4639, 0.4239, 0.4771, 0.3311, 0.4326, 0.5144, 0.8688,
         0.8389, 0.7337, 0.2922, 0.8018, 0.7413, 0.2328, 0.6903, 0.0534, 0.6740,
         0.4394, 0.0058, 0.1914, 0.3602, 0.0023, 0.0429, 0.5230, 0.0794, 0.1046,
         0.5349, 0.8460, 0.5719, 0.1403, 0.7032, 0.4938, 0.5569, 0.4608, 0.2697,
         0.6930, 0.9744, 0.3359, 0.5858, 0.4019, 0.1980, 0.2016, 0.1657]])

In [15]:
# create a mask tensor where the random array is less than 0.15
mask_arr = (rand < 0.15)
mask_arr

tensor([[False, False, False, False, False, False, False, False, False,  True,
         False, False, False, False, False, False, False, False,  True, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True, False, False,  True, False, False,
          True,  True, False,  True,  True, False, False, False,  True, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False]])

In [16]:
# extract the indices of the masked tokens
selection = torch.flatten((mask_arr[0]).nonzero()).tolist()
selection

[9, 18, 34, 37, 40, 41, 43, 44, 48]

We then select these True value positions in the input_ids tensor, and change their values to 103 - we need to copy the original tensor first though, to create a new one called labels:

In [17]:
inputs['labels'] = inputs.input_ids.detach().clone()
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,  2602,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([

And now use mask_arr to mask ~15% of our values:

In [18]:
inputs.input_ids[0, selection] = 103
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,   103,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,   103,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,   103,  2041,  1999,   103,  6863,  2043,
           103,   103,  2749,   103,   103,  7680,  3334,  1999,   103,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([

In [19]:
outputs.loss

In [20]:
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,   103,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,   103,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,   103,  2041,  1999,   103,  6863,  2043,
           103,   103,  2749,   103,   103,  7680,  3334,  1999,   103,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([

And run through the same steps again

In [21]:
outputs = model(**inputs)

In [22]:
# And we will find that we now have an additional output tensor, the loss tensor
outputs.keys()

odict_keys(['loss', 'logits'])

In [23]:
outputs.loss

tensor(1.1912, grad_fn=<NllLossBackward0>)

It is this loss we will be training on